原始数据中一条交易只有一个交易时间，也就是出站时间

采用高德地图API查询站点之间的旅行时间（静态）

In [59]:
import urllib.request #发送请求
from urllib import parse #URL编码
import json #解析json数据
import jsonpath #提取json数据
import pandas as pd #导入pandas库
import numpy as np
import pickle

In [ ]:
stationInfo = pd.read_csv('../../Metro/station_transInfo_cleaned.csv')
stationInfo

In [61]:
dict_staionName = dict(zip(stationInfo['stationID'], stationInfo['name']))
dict_stationLoc = dict(zip(stationInfo['stationID'], zip(stationInfo['lon'],stationInfo['lat'])))
stationID_List = list(stationInfo['stationID'])

travelTime = np.zeros((len(stationID_List), len(stationID_List)))

In [62]:
LBSkey = '5f4a9713e41c83183b72817fb3acb428'
POIkey = '9ad58ff9754b6fe3c7e51f43c06f453f'
mykey = 'd63d7e23659a45277696c954da708d42'

***
I/O Demo

In [ ]:
# Test for 路径规划2.0（地铁图模式）
origin_ID = 2048 
destination_ID = 2049

# 获取 POI
def getPOI(stationID, dict_staionName):
    url = 'https://restapi.amap.com/v5/place/text?'
    params = {
        'key': POIkey,
        'keywords': dict_staionName[stationID],
        'types':150500,
        'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    }

    params = parse.urlencode(params)
    url = url + params
    #print(url)
    response = urllib.request.urlopen(url)
    content = response.read().decode('utf-8')
    data = json.loads(content)
    return data

origin_POI = getPOI(origin_ID, dict_staionName)['pois'][0]['id']
destination_POI = getPOI(destination_ID, dict_staionName)['pois'][0]['id']
print(origin_POI, destination_POI)

In [56]:
# origin = str(dict_stationLoc[origin_ID][0])+','+ str(dict_stationLoc[origin_ID][1])
# destination = str(dict_stationLoc[destination_ID][0])+','+ str(dict_stationLoc[destination_ID][1])

# 获取地铁旅行时间
def getTravelTime(origin_ID, destination_ID, originpoi, destinationpoi, dict_stationLoc):
    url = 'https://restapi.amap.com/v5/direction/transit/integrated?'
    params = {
        'key': LBSkey,
        'originpoi': originpoi,
        'destinationpoi': destinationpoi,
        'origin': str(dict_stationLoc[origin_ID][0])[:9] + ',' + str(dict_stationLoc[origin_ID][1])[:8],
        'destination': str(dict_stationLoc[destination_ID][0])[:9] + ',' + str(dict_stationLoc[destination_ID][1])[:8],
        'strategy': 6,
        'nightflag': 0,
        'date': '2017-08-31',
        'city1': '021',
        'city2': '021',
        'show_fields': 'cost'
    }
    
    params = parse.urlencode(params)
    url = url + params
    #print(url)
    response = urllib.request.urlopen(url)
    content = response.read().decode('utf-8')
    data = json.loads(content)


    return data
        
TravelPlandata = getTravelTime(origin_ID, destination_ID, origin_POI, destination_POI, dict_stationLoc)

In [ ]:
if TravelPlandata['status'] == '1':
    print('成功')
    print('共有方案数量：', len(TravelPlandata['route']['transits']), '个') # TravelPlandata['Count']
    #print('预计距离：', TravelPlandata['route']['transits'][0]['distance'], '米')

print(json.dumps(TravelPlandata, indent=4, ensure_ascii=False))

In [ ]:
# 如果有多个路线，选择时间最少的方案
duration_pool = []
for travelPlan in TravelPlandata['route']['transits']:
    duration_pool.append(travelPlan['cost']['duration'])
min_duration = min(duration_pool)
print( '最短时间：', min_duration, '秒')

***
Debug: 用高德 POI 2.0 会有三个站的POI无法获取，转用1.0

In [ ]:
# Test for 路径规划2.0（地铁图模式）
origin_ID = 2048 
destination_ID = 2049 

# 获取 POI
def getPOIV3(stationID, dict_staionName):
    url='https://restapi.amap.com/v3/place/text?'
    params = {
        'key': POIkey,
        'keywords': dict_staionName[stationID],
        'types':150500,
        'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
        'city':'021',
        
    }

    params = parse.urlencode(params)
    url = url + params
    #print(url)
    response = urllib.request.urlopen(url)
    content = response.read().decode('utf-8')
    data = json.loads(content)
    return data

origin_POI = getPOIV3(origin_ID, dict_staionName)['pois'][0]['id']
destination_POI = getPOIV3(destination_ID, dict_staionName)['pois'][0]['id']
print(origin_POI, destination_POI)

***
### 批处理

In [ ]:
from tqdm import tqdm

dict_stationPOI={}
for i in tqdm(range(len(stationID_List))):
    staionID = stationID_List[i]
    POIFeedback = getPOIV3(staionID, dict_staionName)
    assert POIFeedback['status'] == '1'
    if len(POIFeedback['pois']) == 0:
        print('Null POI for station:', dict_staionName[staionID])
        continue
    dict_stationPOI[staionID] = POIFeedback['pois'][0]['id']
print('Successfully get all POIs')


In [32]:
# 先保存已有的POI，再想办法解决 剩下这三个
with open('TempData/dict_stationPOI.pkl', 'wb') as f:
    pickle.dump(dict_stationPOI, f)

In [ ]:
problematic_stationID = [1631,1226,1241] # 书院站，桂林公园，江浦公园
stationID = 1631 # 书院站
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "书院地铁站",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)

data

In [ ]:
problematic_stationID = [1631,1226,1241] # 书院站，桂林公园，江浦公园
stationID = 1226 # 桂林公园
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "桂林公园（地铁站）",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)

data

In [ ]:
problematic_stationID = [1631,1226,1241] # 书院站，桂林公园，江浦公园
stationID = 1241 # 江浦公园
url='https://restapi.amap.com/v3/place/text?'
params = {
    'key': mykey,
    'keywords': "江浦公园（地铁站）",
    'types':150000,
    'location': str(dict_stationLoc[stationID][0])[:9] + ',' + str(dict_stationLoc[stationID][1])[:8],
    'city':'021',
}
params = parse.urlencode(params)
url = url + params
#print(url)
response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
data = json.loads(content)
data

In [ ]:
# 本质上因为关键词对应POI不唯一
dict_stationPOI[1631] = 'BV10027135'
dict_stationPOI[1226] = 'BV10024842'
dict_stationPOI[1241] = 'BV10039930'

print(len(dict_stationPOI))
with open('TempData/dict_stationPOI.pkl', 'wb') as f:
    pickle.dump(dict_stationPOI, f)

In [ ]:
with open('TempData/dict_stationPOI.pkl', 'rb') as f:
    dict_stationPOI = pickle.load(f)
    
problematic_stationPair = []

# Route Planning
for i in tqdm(range(len(stationID_List))):
    for j in range(i, len(stationID_List)):
        if i==j:
            continue
        origin_ID = stationID_List[i]
        destination_ID = stationID_List[j]
      
        origin_POI = dict_stationPOI[origin_ID]
        destination_POI = dict_stationPOI[destination_ID]
        TravelPlanFeedback = getTravelTime(origin_ID, destination_ID, origin_POI, destination_POI, dict_stationLoc)

        if TravelPlanFeedback['status'] == '1': # 抓取成功
            duration_pool = []   
            for travelPlan  in TravelPlanFeedback['route']['transits']:
                duration_pool.append(travelPlan['cost']['duration'])
            if len(duration_pool) != 0:  # 成功找到路径
                min_duration = min(duration_pool)
                travelTime[i][j] = min_duration
                travelTime[j][i] = min_duration
            else: # 未找到路径
                problematic_stationPair.append((origin_ID, destination_ID))
                print('Problematic Station Pair:', dict_staionName[origin_ID], dict_staionName[destination_ID])
                travelTime[i][j]=-1
                travelTime[j][i]=-1
               
        else:
            problematic_stationPair.append((origin_ID, destination_ID))
            print('Problematic Station Pair:', dict_staionName[origin_ID], dict_staionName[destination_ID])
            travelTime[i][j]=-1
            travelTime[j][i]=-1

print('Successfully get all travel time')